In [2]:
import pandas as pd
from haversine import haversine


In [3]:
df = pd.read_csv('simplemaps_uscities_basicv1.74/uscities.csv')
df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,Gross,Gross,NE,Nebraska,31015,Boyd,42.9461,-98.5697,2,6,polygon,False,True,America/Chicago,3,68719,1840011032
28334,Lotsee,Lotsee,OK,Oklahoma,40143,Tulsa,36.1334,-96.2091,2,39,polygon,False,True,America/Chicago,3,74063,1840021674
28335,The Ranch,The Ranch,MN,Minnesota,27087,Mahnomen,47.3198,-95.6952,2,2,polygon,False,True,America/Chicago,3,56557,1840039629
28336,Shamrock,Shamrock,OK,Oklahoma,40037,Creek,35.9113,-96.5772,2,2,polygon,False,True,America/Chicago,3,74068,1840022701


In [4]:
cities = pd.read_csv('cities.csv', header=None)
cities = list(map(str.lower, cities[0]))
len(cities), cities

(41,
 ['phoenix',
  'little rock',
  'san francisco',
  'denver',
  'hartford',
  'wilmington',
  'washington',
  'jacksonville',
  'miami',
  'atlanta',
  'chicago',
  'indianapolis',
  'des moines',
  'wichita',
  'louisville',
  'new orleans',
  'baltimore',
  'detroit',
  'minneapolis',
  'kansas city',
  'st. louis',
  'omaha',
  'albuquerque',
  'albany',
  'buffalo',
  'cincinnati',
  'cleveland',
  'columbus',
  'philadelphia',
  'pittsburgh',
  'providence',
  'memphis',
  'nashville',
  'dallas',
  'houston',
  'salt lake city',
  'norfolk',
  'richmond',
  'seattle',
  'charleston',
  'milwaukee'])

In [5]:
by_city = df.set_index(['city'])
by_city = by_city.loc[filter(lambda name: name.lower() in cities, by_city.index)].reset_index()

# disamiguate cities using max population
idx = by_city.groupby(['city'])['population'].transform(max) == by_city['population']

# prettify found data
by_city = by_city[idx][['city', 'population', 'lat', 'lng']].drop_duplicates()

# by_city.to_csv('out.csv')
by_city = by_city.set_index('city')
len(by_city) # should be 41 as in cities.csv

41

In [15]:
# calc distance to chicago
chicago = by_city.loc['Chicago']
by_city['km_to_chicago'] = by_city.apply(lambda city: haversine((city.lat, city.lng), (chicago.lat, chicago.lng)), axis=1).round()

In [14]:
by_city['population_2020'] = (by_city['population'] / 1000).round()
out = by_city.drop('population', axis=1)
out


,lat,lng,km_to_chicago,population_2020
city,,,,
Chicago,41.8373,-87.6862,0.000000,8604.0
Miami,25.7839,-80.2102,1912.053590,6446.0
Dallas,32.7936,-96.7662,1285.252179,5744.0
Philadelphia,40.0077,-75.1339,1073.035756,5649.0
Houston,29.7863,-95.3889,1507.729598,5464.0
Atlanta,33.7627,-84.4224,942.330526,5449.0
Washington,38.9047,-77.0163,960.119229,5379.0
Phoenix,33.5722,-112.0891,2324.834948,4220.0
Seattle,47.6211,-122.3244,2785.995651,3789.0
